# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [12]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [13]:
# Setup the Database
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()
cur.execute("DROP DATABASE IF EXISTS sparkifydb")
cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")
conn.commit()
conn.close()

In [14]:
# Get a connection and cursor
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
# Then create the tables
for query in create_table_queries:
        cur.execute(query)
        conn.commit()
conn.commit()

In [15]:
# Traverse path given recursively and find json files
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [16]:
# Well, lets get those files and their absolute path
song_files = get_files('data/song_data')
# Check that we have results by printing the first element, should not be empty
print(song_files[0])

/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json


In [17]:
# For this notebook, we need only 1 file to analyze
filepath = song_files[0]

In [18]:
# Open the file and load the content into a dataframe, print the item
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


In [19]:
# Now run again for all files
df = pd.DataFrame()

# Traverse the file list and add new lines into dataframe
for path in song_files:
    new_line = pd.read_json(path, lines=True)
    df = df.append(new_line, ignore_index=True)
   
# Remove duplicates and print some data
df = df.drop_duplicates()
df.head(5)

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008
1,AR558FS1187FB45658,NaN,,NaN,40 Grit,75.67628,1,SOGDBUF12A8C140FAA,Intro,2003
2,ARVBRGZ1187FB4675A,NaN,,NaN,Gwen Stefani,290.55955,1,SORRZGD12A6310DBC3,Harajuku Girls,2004
3,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994
4,AREVWGE1187B9B890A,-13.44200,Noci (BA),-41.99520,Bitter End,282.43546,1,SOFCHDR12AB01866EF,Living Hell,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [20]:
# Select columns from df
songs_table_columns = df[["song_id","title","artist_id","year","duration"]]
# Select only the values from dataframe
songs_table_values = songs_table_columns.values
# Select the first record at Index 0
print(songs_table_values[0])
print(type(songs_table_values))
# Convert array to list and print first item
song_data = list(songs_table_columns.values)
print(type(song_data))

['SOINLJW12A8C13314C' 'City Slickers' 'AR8IEZO1187B99055E' 2008 149.86404]
<class 'numpy.ndarray'>
<class 'list'>


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [21]:
# Insert a single record only first
cur.execute(song_table_insert, songs_table_values[0])
conn.commit()

In [12]:
# Now instead of just inserting a single value like above, put the whole df with song data into the table
# See if it is there and clear the table again
cur.execute("DROP TABLE IF EXISTS songs;")
cur.execute("CREATE TABLE IF NOT EXISTS songs (song_id VARCHAR PRIMARY KEY, title VARCHAR, artist_id VARCHAR, year INT, duration FLOAT);")
conn.commit()

for song in songs_table_values:
    cur.execute(song_table_insert, song)
    conn.commit()

# Check if data is in table
cur.execute("SELECT * FROM songs;")
cur.fetchone()

('SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
artist_data = artist_data.drop_duplicates()
artist_table_values = artist_data.values
artist_table_index = artist_data.index
artist_data = artist_table_values.tolist()
print(artist_data[0])
print(type(artist_data))

['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]
<class 'list'>


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data[0])
conn.commit()

In [15]:
# Now instead of just inserting a single value like above, put the whole df with artist data into the table
# See if it is there and clear the table again
cur.execute("DROP TABLE IF EXISTS artists;")
cur.execute("CREATE TABLE IF NOT EXISTS artists (artist_id VARCHAR PRIMARY KEY, name VARCHAR, location VARCHAR, latitude FLOAT, longitude FLOAT);")
conn.commit()

for artist in artist_data:
    cur.execute(artist_table_insert, artist)
    conn.commit()

# Check if data is in table
cur.execute("SELECT * FROM artists;")
cur.fetchone()

('AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files('data/log_data')
print(log_files)

['/home/workspace/data/log_data/2018/11/2018-11-29-events.json', '/home/workspace/data/log_data/2018/11/2018-11-27-events.json', '/home/workspace/data/log_data/2018/11/2018-11-11-events.json', '/home/workspace/data/log_data/2018/11/2018-11-18-events.json', '/home/workspace/data/log_data/2018/11/2018-11-08-events.json', '/home/workspace/data/log_data/2018/11/2018-11-30-events.json', '/home/workspace/data/log_data/2018/11/2018-11-17-events.json', '/home/workspace/data/log_data/2018/11/2018-11-20-events.json', '/home/workspace/data/log_data/2018/11/2018-11-28-events.json', '/home/workspace/data/log_data/2018/11/2018-11-06-events.json', '/home/workspace/data/log_data/2018/11/2018-11-23-events.json', '/home/workspace/data/log_data/2018/11/2018-11-13-events.json', '/home/workspace/data/log_data/2018/11/2018-11-21-events.json', '/home/workspace/data/log_data/2018/11/2018-11-16-events.json', '/home/workspace/data/log_data/2018/11/2018-11-04-events.json', '/home/workspace/data/log_data/2018/11/

In [17]:
# Select first log file only
filepath = log_files[0]
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-29-events.json


In [18]:
# Read and print content as stated above
df = pd.read_json(filepath, lines=True)
# Result Control
df.head(1)
print(df.size)

6588


In [19]:
# Now lets traverse all files and add to a new df
for path in log_files:
    new_line = pd.read_json(path, lines=True)
    df = df.append(new_line, ignore_index=True)
   
# Remove duplicates and print some data
df = df.drop_duplicates()
# Result Control
print(df.size)
df.head(5)

145026


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [23]:
# Filter Data for next song action and sort ascending
times = df.query('page == "NextSong"')
t = pd.to_datetime(times.ts, unit='ms').sort_values()

# Extract columns with milliseconds and put into new series as Timestamps

# Create other columns from timestamps
#hours = list(timestamps.dt.hour.values)
#days = list(timestamps.dt.day.values)
#weeks = list(timestamps.dt.week.values)
#months = list(timestamps.dt.month.values)
#years = list(timestamps.dt.year.values)
#weekdays = list(timestamps.dt.weekday.values)
#timestamps = list(timestamps.dt.values)

# List of column lables
column_labels= ("timestamp", "hour", "day", "week", "month", "year", "weekday")

time_data = (t.dt.values, t.dt.hour.values, t.dt.day.values, t.dt.week.values, t.dt.month.values, t.dt.year.values, t.dt.weekday.values)
column_labels = ("timestamp", "hour", "day", "week", "month", "year", "weekday")
dict_ts = dict(zip(column_labels, time_data))
time_df = pd.DataFrame.from_dict(dict_ts)

time_df = time_df.drop_duplicates()
# Create dictionary
#dict_ts = {'timestamp': timestamps, 'hour': hours, 'day': days, 'week': weeks, 'month': months, 'year': years, 'weekdays': weekdays}
#Create dataframe from dictionary
#time_df = pd.DataFrame.from_dict(dict_ts)
#time_df = time_df.drop_duplicates()
# Result Control
time_df.head()

,timestamp,hour,day,week,month,year,weekday
6179,2018-11-01 21:01:46.796,21,1,44,11,2018,3
6181,2018-11-01 21:05:52.796,21,1,44,11,2018,3
6182,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6183,2018-11-01 21:11:13.796,21,1,44,11,2018,3
6184,2018-11-01 21:17:33.796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
# Clear the table
cur.execute("DROP TABLE IF EXISTS time;")
cur.execute("CREATE TABLE IF NOT EXISTS time (start_time TIMESTAMP PRIMARY KEY, hour INT, day INT, week INT, month INT, year INT, weekday INT)")
conn.commit()
# Insert the time data
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:
# Select the data, remove data without userId
user_df =  df[["userId", "firstName", "lastName", "gender", "level"]]
# Convert user ids to numbers
user_df['userId'] = pd.to_numeric(user_df['userId'], errors='ignore', downcast='integer')
# Remove NaNs, duplicates and sort data
user_df.dropna(axis=0, inplace=True)
user_df = user_df.drop_duplicates()
user_df = user_df.sort_values(by='userId')
user_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,userId,firstName,lastName,gender,level
1934,2.0,Jizelle,Benjamin,F,free
7070,3.0,Isaac,Valdez,M,free
42,4.0,Alivia,Terrell,F,free
767,5.0,Elijah,Davis,M,free
1724,6.0,Cecilia,Owens,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
conn.commit()
cur.execute("DROP TABLE users;")
cur.execute("CREATE TABLE IF NOT EXISTS users (user_id INT, first_name VARCHAR, last_name VARCHAR, gender VARCHAR, level VARCHAR);")
conn.commit()
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
# We could try and get the song selected via SQL and JOIN like here:
song_select = "SELECT s.song_id, a.artist_id FROM songs s JOIN artists a ON s.artist_id = a.artist_id WHERE s.title LIKE %s AND a.name LIKE %s AND s.duration = %s;"
songplay_table_insert = "INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

for index, row in df.iterrows():
    selectors = (row.song, row.artist, row.length)
    cur.execute(song_select, selectors)
    results = cur.fetchone()

    # If it yields any results, complete the dataset and insert it into the table
    if results:
        songid, artistid = results
        # convert the timestamps to datetime
        row.ts = pd.to_datetime(row.ts, unit='ms')
        songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
    else:
        songid, artistid = None, None
        
# Since I kept getting a string conversion exception, probably because of the single quote '' characters in the song titles
# I just approached it in Pandas also
# Get the songs table
song_table = "SELECT s.song_id, s.artist_id, s.title, s.duration FROM songs s;"
cur.execute(song_table)
songs_list = pd.DataFrame(cur.fetchall(), columns=('songid', 'songs_artistid', 'title', 'duration'))
# Get the artists table
artist_table = "SELECT a.name, a.artist_id FROM artists a;"
cur.execute(artist_table)
art_list = pd.DataFrame(cur.fetchall(), columns=['name', 'artists_artistid'])
# Join them on artist id
art_list = songs_list.join(art_list.set_index('artists_artistid'), on='songs_artistid')
# Now join this table into the songplay data in dataframe df
songplay_data = df.join(art_list.set_index('title'), on='song')
songplay_data = songplay_data[['ts', 'userId', 'level', 'songid', 'songs_artistid', 'sessionId', 'location', 'userAgent']]
# And remove the NaN lines to see the matched results
not_empty = songplay_data.songid.notna()
songplay_data[not_empty].head()
# --> But we are not writing this to the table, just remember the 2 wasted days
   

,ts,userId,level,songid,songs_artistid,sessionId,location,userAgent
1006,1543358159796,80,paid,SOGDBUF12A8C140FAA,AR558FS1187FB45658,992,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4216,1542837407796,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
6504,1542171963796,10,free,SOGDBUF12A8C140FAA,AR558FS1187FB45658,484,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
8237,1542618860796,24,paid,SOGDBUF12A8C140FAA,AR558FS1187FB45658,672,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [72]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.